Basic Imports

In [2]:
import os
import sys
import argparse
import cv2
import time
import torch
from vidgear.gears import CamGear
import numpy as np

# HRNet Directory
sys.path.append('models/simple-HigherHRNet')

# Import the required HRNet model classes and utilities
from SimpleHigherHRNet import SimpleHigherHRNet
from misc.visualization import draw_points, draw_skeleton, draw_points_and_skeleton, joints_dict, check_video_rotation
from misc.utils import find_person_id_associations

Testing Camera


In [ ]:
import cv2

cam0 = cv2.VideoCapture(0)
cam1 = cv2.VideoCapture(1)
cam2 = cv2.VideoCapture(2)

while True:
    ret1, frame1 = cam0.read()
    ret2, frame2 = cam2.read()

    frame1 = cv2.flip(frame1, 1)
    frame2 = cv2.flip(frame2, 1)

    if(ret1):
        cv2.imshow('cam1', frame1)    
    if(ret2):
        cv2.imshow('cam2', frame2)  
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break      
cam1.release()
cam2.release()
cv2.destroyAllWindows()

# Capture Images

Press 'q' to quit, 'c' to capture images, 't' to capture a specified number of images

In [ ]:
import cv2
from pathlib import Path
from datetime import datetime, timedelta

pathname = input("Please input the desired path name")

# Initialize video capture for two cameras
cam1 = cv2.VideoCapture(1)
cam2 = cv2.VideoCapture(2)

# Create directories for saving images
data_dir = Path(pathname)
cam1_dir = data_dir / 'cam1'
cam2_dir = data_dir / 'cam2'
cam1_dir.mkdir(parents=True, exist_ok=True)
cam2_dir.mkdir(parents=True, exist_ok=True)

# Capture settings
print("Press 'q' to quit, 'c' to capture images, 't' to capture a specified number of images")
num_images_to_capture = int(input("How many pictures do you want to take? "))

# img_count = 0  # Image counter to synchronize images from both cameras
# capture_delay = timedelta(seconds=0.1)  # Time between captures

def capture_and_save_images(img_count):
    """Captures and saves images from both cameras with a timestamp."""
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    cam1_filename = cam1_dir / f'cam1_{timestamp}_{img_count:04d}.png'
    cam2_filename = cam2_dir / f'cam2_{timestamp}_{img_count:04d}.png'

    ret1, frame1 = cam1.read()
    ret2, frame2 = cam2.read()

    frame1 = cv2.flip(frame1, 1)
    frame2 = cv2.flip(frame2, 1)

    if ret1 and ret2:
        cv2.imwrite(str(cam1_filename), frame1)
        cv2.imwrite(str(cam2_filename), frame2)
        print(f"Captured: {cam1_filename} and {cam2_filename}")
    else:
        print("Failed to capture image from one or both cameras.")

frame_counter = 100 # for the file name
is_capturing = False

images_captured = 0 # counting the pictures

last_capture_time = 0 # couuting the time interval, so each 0.1 second one picture will be taken

while True:
    ret1, frame1 = cam1.read()
    ret2, frame2 = cam2.read()

    frame1 = cv2.flip(frame1, 1)
    frame2 = cv2.flip(frame2, 1)

    if ret1:
        cv2.imshow('Camera 1', frame1)
    if ret2:
        cv2.imshow('Camera 2', frame2)
    # Capture image if in capture mode
    current_time = time.time() # get the current time in whlie loop
    if is_capturing and (current_time - last_capture_time) >= 0.001: # capture interval default set as 0.1 second

        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        cam1_filename = cam1_dir / f'cam1_{timestamp}_{images_captured:04d}.png'
        cam2_filename = cam2_dir / f'cam2_{timestamp}_{images_captured:04d}.png'

        cv2.imwrite(str(cam1_filename), frame1)
        cv2.imwrite(str(cam2_filename), frame2)
        

        frame_counter += 1
        images_captured += 1
        last_capture_time = current_time
        print(f"Captured: {cam1_filename} and {cam2_filename}")

        if images_captured >= num_images_to_capture: # stoping conditions if the taken pipcutures exceed the expected numbers
            is_capturing = False
            print("Finished capturing images.")

    key = cv2.waitKey(10) & 0xFF

    if key == ord('q'):  # Quit the program
        break
    elif key == ord('c'):  # Capture one set of images
        capture_and_save_images(images_captured)
        images_captured += 1
    elif key == ord('t'):  # Press 't' to start capturing images
        
        # capture_interval = float(input("Enter the interval between captures (in seconds): "))
        # print(f"Capturing {num_images_to_capture} images at {capture_interval} second intervals. Get ready to pose!")
        is_capturing = True
        images_captured = 0
        last_capture_time = time.time()
        # time.sleep(5)

# Release the camera resources and close windows
cam1.release()
cam2.release()
cv2.destroyAllWindows()


img -> pose estimation

In [7]:
import cv2
import torch
from SimpleHigherHRNet import SimpleHigherHRNet
from misc.visualization import draw_points_and_skeleton, joints_dict
import matplotlib.pyplot as plt
from os import listdir
from pathlib import Path

pathname = input("Please input the path name (for pose estimation)")
pathname = Path(pathname)

#initializing the model
model = SimpleHigherHRNet(
    c=48,
    nof_joints=17,
    checkpoint_path="models/simple-HigherHRNet/weights/pose_higher_hrnet_w48_640.pth.tar",
    resolution=640,
    return_bounding_boxes=False,
    max_nof_people=30,
    max_batch_size=16,
    device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
)
print("Model initialized successfully.")

#images path
folder_cam1 = pathname / "cam1"
folder_cam2 = pathname / "cam2"

#new path for pose estimation
# data_dir = Path('data_bruh_jack')
cam1_new_dir = pathname / 'cam1_pose'
cam2_new_dir = pathname / 'cam2_pose'

cam1_new_dir.mkdir(parents=True, exist_ok=True)
cam2_new_dir.mkdir(parents=True, exist_ok=True)

def pose_estimate(image_dir, output_dir):
    #Create a new txt file storing the joints
    f = open(f"{image_dir}.txt", "w")

    for image_cam in listdir(image_dir):
        image_path = os.path.join(image_dir, image_cam)
        image = cv2.imread(image_path)

        # Check if the image was loaded correctly
        if image is None:
            raise ValueError(f"Image at path '{image_path}' could not be loaded. Please check the path.")
        print("Image loaded successfully.")

        #predict the joints
        joints = model.predict(image)
        
        # Check if joints were detected
        if len(joints) == 0:
            print("No joints detected.")
        else:
            print(f"Detected joints for {len(joints)} people.")

        # Draw the keypoints and skeleton on the image
        if len(joints) > 0:
            skeleton = joints_dict()['coco']['skeleton']
            image = draw_points_and_skeleton(image, joints[0], skeleton, points_color_palette='gist_rainbow', skeleton_color_palette='jet')

        #image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) <- for cv2.imshow

        cam_filename = output_dir / image_cam

        cv2.imwrite(str(cam_filename), image)

        #Store the keypoints
        joints_for_superglue = [[int(point[1]), int(point[0])] for point in joints[0] if point[2] >= 0.1]
        

        print(f"image name: {image_path}, joints_for_superglue: {joints_for_superglue}")
        input = str(cam_filename) + " " + str(joints_for_superglue) + "\n"
        # print(input)
        f.write(input)
    
    
    f.close()

#iterate through the image path
pose_estimate(folder_cam1, cam1_new_dir)
pose_estimate(folder_cam2, cam2_new_dir)

device: 'cuda' - 1 GPU(s) will be used
Model initialized successfully.
Image loaded successfully.
Detected joints for 1 people.
image name: data\cam1\cam1_20240912_120234_0001.png, joints_for_superglue: [[296, 81], [299, 75], [290, 72], [266, 71], [289, 133], [245, 120], [298, 196], [238, 184], [314, 240], [222, 248], [301, 244], [268, 247], [298, 342], [266, 345], [292, 429], [266, 446]]
Image loaded successfully.
Detected joints for 1 people.
image name: data\cam1\cam1_20240912_120234_0002.png, joints_for_superglue: [[296, 81], [299, 75], [290, 72], [266, 71], [290, 133], [246, 119], [298, 195], [237, 183], [315, 240], [220, 248], [301, 245], [267, 249], [298, 344], [266, 346], [292, 429], [267, 447]]
Image loaded successfully.
Detected joints for 1 people.
image name: data\cam1\cam1_20240912_120235_0003.png, joints_for_superglue: [[295, 80], [298, 75], [289, 71], [266, 71], [289, 132], [245, 119], [298, 195], [236, 184], [316, 240], [218, 248], [302, 246], [266, 252], [299, 344], [2